<a href="https://colab.research.google.com/github/adolfolh/casuality-classification/blob/main/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run to download data
!wget https://github.com/adolfolh/casuality-classification/raw/main/data/casualties-2020.csv

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px

In [4]:
# Import data
raw_data = pd.read_csv("casualties-2020.csv", dtype={
    "accident_index" : "string",
    "accident_reference" : "string",
    "casualty_class" : "category",
    "sex_of_casualty" : "category",
    "age_band_of_casualty" : "category",
    "casualty_severity" : "category",
    "pedestrian_location" : "category",
    "pedestrian_movement" : "category",
    "car_passenger" : "category",
    "bus_or_coach_passenger" : "category",
    "pedestrian_road_maintenance_worker" : "category",
    "casualty_type" : "category",
    "casualty_home_area_type" : "category",
    "casualty_imd_decile" : "category"
    })

In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115584 entries, 0 to 115583
Data columns (total 18 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   accident_index                      115584 non-null  string  
 1   accident_year                       115584 non-null  int64   
 2   accident_reference                  115584 non-null  string  
 3   vehicle_reference                   115584 non-null  int64   
 4   casualty_reference                  115584 non-null  int64   
 5   casualty_class                      115584 non-null  category
 6   sex_of_casualty                     115584 non-null  category
 7   age_of_casualty                     115584 non-null  int64   
 8   age_band_of_casualty                115584 non-null  category
 9   casualty_severity                   115584 non-null  category
 10  pedestrian_location                 115584 non-null  category
 11  pedestrian_mo